##### Biblioteki

In [ ]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.decomposition import PCA


##### Wczytywanie plików

In [ ]:
file_bezrobocie = "raw data/stopa_bezrobocia.xlsx"
file_przestepstwa_kryminalne = "raw data/przestepstwa_kryminalne.xlsx"
file_przestepstwa_gospodarcze = "raw data/przestepstwa_gospodarcze.xlsx"
file_przestepstwa_drogowe = "raw data/przestepstwa_drogowe.xlsx"

In [ ]:
df_bezrobocie = pd.read_excel(file_bezrobocie)
df_kr = pd.read_excel(file_przestepstwa_kryminalne)
df_gosp = pd.read_excel(file_przestepstwa_gospodarcze)
df_dro = pd.read_excel(file_przestepstwa_drogowe)


#### Przygotowywanie danych

##### Średnie bezrobocie rocznie

In [ ]:
df_bezrobocie_roczne = df_bezrobocie.groupby("Rok")["Wartość"].mean().reset_index()
df_bezrobocie_roczne.columns = ["Rok", "Bezrobocie [%]"]
df_bezrobocie_roczne.drop([31,32,33], axis=0, inplace=True)
df_bezrobocie_roczne["Bezrobocie [%]"] = df_bezrobocie_roczne["Bezrobocie [%]"].round(2)
df_bezrobocie_roczne["Rok"] = pd.to_datetime(df_bezrobocie_roczne["Rok"], format="%Y")
df_bezrobocie_roczne["Rok"] = df_bezrobocie_roczne["Rok"].dt.year
df_bezrobocie_roczne


##### Przygotowanie odpowednio Dataframów z przestępstwami

In [ ]:
df_kr = df_kr[["Jednostka podziału administracyjnego", "Rok", "Przestępstwa stwierdzone"]]
df_kr_Polska = df_kr[df_kr["Jednostka podziału administracyjnego"] == "Polska"].copy()
df_kr_Polska.columns = ["Kraj", "Rok", "Liczba przestępstw"]
df_kr_Polska.drop(columns="Kraj", inplace=True)

In [ ]:
df_gosp = df_gosp[["Jednostka podziału administracyjnego", "Rok", "Przestępstwa stwierdzone"]]
df_gosp_Polska = df_gosp[df_gosp["Jednostka podziału administracyjnego"] == "Polska"].copy()
df_gosp_Polska.columns = ["Kraj", "Rok", "Liczba przestępstw"]
df_gosp_Polska.drop(columns="Kraj", inplace=True)

In [ ]:
df_dro = df_dro[["Jednostka podziału administracyjnego", "Rok", "Przestępstwa stwierdzone"]]
df_dro_Polska = df_dro[df_dro["Jednostka podziału administracyjnego"] == "Polska"].copy()
df_dro_Polska.columns = ["Kraj", "Rok", "Liczba przestępstw"]
df_dro_Polska.drop(columns="Kraj", inplace=True)

In [ ]:
# df_przestepstwa_kryminalne = df_przestepstwa_kryminalne.rename(columns={"Jednostka podziału administracyjnego" : "Region"})

# i = df_przestepstwa_kryminalne[((df_przestepstwa_kryminalne.Region == "Polska"))].index

# df_przestepstwa_kryminalne.drop(i, inplace=True)
# df_przestepstwa_kryminalne.reset_index(drop=True, inplace=True)
# df_przestepstwa_kryminalne

#### Eksploracyjna analiza danych (EDA)

1. Bezrobocie - statystyka opisowa

In [ ]:
średnia_df_bezrobocie_roczne = df_bezrobocie_roczne["Bezrobocie [%]"].mean().round(2)
mediana_df_bezrobocie_roczne = df_bezrobocie_roczne["Bezrobocie [%]"].median().round(2)
odchylenie_standardowe_df_bezrobocie_roczne = df_bezrobocie_roczne["Bezrobocie [%]"].std().round(2)
wariancja_df_bezrobocie_roczne = df_bezrobocie_roczne["Bezrobocie [%]"].var().round(2)
cv = odchylenie_standardowe_df_bezrobocie_roczne/średnia_df_bezrobocie_roczne #współczynnik zmienności
cv = cv.round(2)

print(f"Srednie bezrobocie wynosi: {średnia_df_bezrobocie_roczne}")
print(f"Mediana dla bezrobocia: {mediana_df_bezrobocie_roczne}")
print(f"Odchylenie standardowe dla bezrobocia: {odchylenie_standardowe_df_bezrobocie_roczne}")
print(f"Wariancja dla bezrobocia: {wariancja_df_bezrobocie_roczne}")
print(f"Współczynnik zmienności wynosi: {cv}")

1.1 Bezrobocie - wykresy

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df_bezrobocie_roczne["Rok"], df_bezrobocie_roczne["Bezrobocie [%]"], marker='o', linestyle='-', color='blue', linewidth=2, label="Bezrobocie [%]")
plt.axhline(średnia_df_bezrobocie_roczne, color="red", linestyle="--", linewidth=2, label=f"Średnie bezrobocie: {średnia_df_bezrobocie_roczne:.2f}%")
plt.title("Bezrobocie w latach 1999-2021", fontsize=20)
plt.xlabel("Rok", fontsize=16)
plt.ylabel("Stopa bezrobocia [%]", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=12)
plt.show()

In [ ]:
#Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_bezrobocie_roczne["Bezrobocie [%]"], kde=True, bins=30)
plt.title("Rozkład stopy bezrobocia")
plt.xlabel("Bezrobocie [%]")
plt.ylabel("Liczba przypadków (Częstość)")
plt.show()

In [ ]:
#Obliczenie linii trendu
z = np.polyfit(df_bezrobocie_roczne["Rok"], df_bezrobocie_roczne["Bezrobocie [%]"], 1)
p = np.poly1d(z)

# Wykres stopy bezrobocia z linią trendu
plt.figure(figsize=(15, 5))
plt.plot(df_bezrobocie_roczne["Rok"], df_bezrobocie_roczne["Bezrobocie [%]"], label="Bezrobocie [%]", color="blue", marker="o")
plt.plot(df_bezrobocie_roczne["Rok"], p(df_bezrobocie_roczne["Rok"]), linestyle="--", color="red", label="Linia trendu")
plt.title("Bezrobocie w latach 1999-2021 z linią trendu")
plt.xlabel("Rok")
plt.ylabel("Bezrobocie [%]")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.show()

2. Przestępstwa kryminalne

In [ ]:
średnia_df_kr_Polska = df_kr_Polska["Liczba przestępstw"].mean().round(2)
mediana_df_kr_Polska = df_kr_Polska["Liczba przestępstw"].median().round(2)
odchylenie_standardowe_df_kr_Polska = df_kr_Polska["Liczba przestępstw"].std().round(2)
wariancja_df_kr_Polska = df_kr_Polska["Liczba przestępstw"].var().round(2)
cv = odchylenie_standardowe_df_kr_Polska/średnia_df_kr_Polska #współczynnik zmienności
cv = cv.round(2)

print(f"Srednie liczba przestępstw kryminalnych wynosi: {średnia_df_kr_Polska}")
print(f"Mediana dla liczby przestępstw kryminalnych wynosi: {mediana_df_kr_Polska}")
print(f"Odchylenie standardowe dla liczby przestępstw kryminalnych: {odchylenie_standardowe_df_kr_Polska}")
print(f"Wariancja dla liczby przestępstw kryminalnych: {wariancja_df_kr_Polska}")
print(f"Współczynnik zmienności wynosi: {cv}")

2.2 Przestępstaw kryminalne - wykresy

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df_kr_Polska["Rok"], df_kr_Polska["Liczba przestępstw"])
plt.axhline(średnia_df_kr_Polska, color = "red", linestyle = "--", linewidth=2, label=f"Srednie liczba przestępstw: {średnia_df_kr_Polska:.2f}%")
plt.title("Liczba przestępstw kryminalnych w latach 1990-2021", fontsize=20)
plt.xlabel("Rok", fontsize=16)
plt.ylabel("Stopa bezrobocia [%]", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=12)
plt.show()

In [ ]:
#Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_kr_Polska["Liczba przestępstw"], kde=True, bins=30)
plt.title("Rozkład liczby przestępstw kryminalnych")
plt.xlabel("Liczba przestępstw")
plt.ylabel("Liczba przypadków (Częstość)")
plt.show()

In [ ]:
#Obliczenie linii trendu
z = np.polyfit(df_kr_Polska["Rok"], df_kr_Polska["Liczba przestępstw"], 1)
p = np.poly1d(z)

# Wykres stopy bezrobocia z linią trendu
plt.figure(figsize=(15, 5))
plt.plot(df_kr_Polska["Rok"], df_kr_Polska["Liczba przestępstw"], label="Liczba przestępstw", color="blue", marker="o")
plt.plot(df_kr_Polska["Rok"], p(df_kr_Polska["Rok"]), linestyle="--", color="red", label="Linia trendu")
plt.title("Liczba przestępstw kryminalnych w latach 1999-2021 z linią trendu")
plt.xlabel("Rok")
plt.ylabel("Liczba przestępstw")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.show()

3. Przestępstwa gospodarcze

In [ ]:
średnia_df_gosp_Polska = df_gosp_Polska["Liczba przestępstw"].mean().round(2)
mediana_df_gosp_Polska = df_gosp_Polska["Liczba przestępstw"].median().round(2)
odchylenie_standardowe_df_gosp_Polska = df_gosp_Polska["Liczba przestępstw"].std().round(2)
wariancja_df_gosp_Polska = df_gosp_Polska["Liczba przestępstw"].var().round(2)
cv = odchylenie_standardowe_df_gosp_Polska/średnia_df_gosp_Polska #współczynnik zmienności
cv = cv.round(2)

print(f"Srednie liczba przestępstw gospodarczych wynosi: {średnia_df_gosp_Polska}")
print(f"Mediana dla liczby przestępstw gospodarczych wynosi: {mediana_df_gosp_Polska}")
print(f"Odchylenie standardowe dla liczby przestępstw gospodarczych: {odchylenie_standardowe_df_gosp_Polska}")
print(f"Wariancja dla liczby przestępstw gospodarczych: {wariancja_df_gosp_Polska}")
print(f"Współczynnik zmienności wynosi: {cv}")

3.1. Przestępstwa gospodarcze - wykresy

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df_gosp_Polska["Rok"], df_gosp_Polska["Liczba przestępstw"])
plt.axhline(średnia_df_gosp_Polska, color = "red", linestyle = "--", linewidth=2, label=f"Srednie liczba przestępstw: {średnia_df_gosp_Polska:.2f}%")
plt.title("Liczba przestępstw gospodarczych w latach 1999-2021", fontsize=20)
plt.xlabel("Rok", fontsize=16)
plt.ylabel("Stopa bezrobocia [%]", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=12)
plt.show()

In [ ]:
#Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_gosp_Polska["Liczba przestępstw"], kde=True, bins=30)
plt.title("Rozkład liczby przestępstw gospodarczych")
plt.xlabel("Liczba przestępstw")
plt.ylabel("Liczba przypadków (Częstość)")
plt.show()

In [ ]:
#Obliczenie linii trendu
z = np.polyfit(df_gosp_Polska["Rok"], df_gosp_Polska["Liczba przestępstw"], 1)
p = np.poly1d(z)

# Wykres stopy bezrobocia z linią trendu
plt.figure(figsize=(15, 5))
plt.plot(df_gosp_Polska["Rok"], df_gosp_Polska["Liczba przestępstw"], label="Liczba przestępstw", color="blue", marker="o")
plt.plot(df_gosp_Polska["Rok"], p(df_gosp_Polska["Rok"]), linestyle="--", color="red", label="Linia trendu")
plt.title("Liczba przestępstw gospodarczych w latach 1999-2021 z linią trendu")
plt.xlabel("Rok")
plt.ylabel("Liczba przestępstw")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.show()

4. Przestępstwa drogowe

In [ ]:
średnia_df_dro_Polska = df_dro_Polska["Liczba przestępstw"].mean().round(2)
mediana_df_dro_Polska = df_dro_Polska["Liczba przestępstw"].median().round(2)
odchylenie_standardowe_df_dro_Polska = df_dro_Polska["Liczba przestępstw"].std().round(2)
wariancja_df_dro_Polska = df_dro_Polska["Liczba przestępstw"].var().round(2)
cv = odchylenie_standardowe_df_dro_Polska/średnia_df_dro_Polska #współczynnik zmienności
cv = cv.round(2)

print(f"Srednie liczba przestępstw drogowych wynosi: {średnia_df_dro_Polska}")
print(f"Mediana dla liczby przestępstw drogowych wynosi: {mediana_df_dro_Polska}")
print(f"Odchylenie standardowe dla liczby drogowych gospodarczych: {odchylenie_standardowe_df_dro_Polska}")
print(f"Wariancja dla liczby przestępstw drogowych: {wariancja_df_dro_Polska}")
print(f"Współczynnik zmienności wynosi: {cv}")

4.1 Przestępstwa drogowe - wykresy

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df_dro_Polska["Rok"], df_dro_Polska["Liczba przestępstw"])
plt.axhline(średnia_df_dro_Polska, color = "red", linestyle = "--", linewidth=2, label=f"Srednie liczba przestępstw: {średnia_df_dro_Polska:.2f}%")
plt.title("Liczba przestępstw drogowych w latach 1999-2021", fontsize=20)
plt.xlabel("Rok", fontsize=16)
plt.ylabel("Stopa bezrobocia [%]", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(fontsize=12)
plt.show()

In [ ]:
#Histogram
plt.figure(figsize=(10, 5))
sns.histplot(df_dro_Polska["Liczba przestępstw"], kde=True, bins=30)
plt.title("Rozkład liczby przestępstw drogowych")
plt.xlabel("Liczba przestępstw")
plt.ylabel("Liczba przypadków (Częstość)")
plt.show()

In [ ]:
#Obliczenie linii trendu
z = np.polyfit(df_dro_Polska["Rok"], df_dro_Polska["Liczba przestępstw"], 1)
p = np.poly1d(z)

# Wykres stopy bezrobocia z linią trendu
plt.figure(figsize=(15, 5))
plt.plot(df_dro_Polska["Rok"], df_dro_Polska["Liczba przestępstw"], label="Liczba przestępstw", color="blue", marker="o")
plt.plot(df_dro_Polska["Rok"], p(df_dro_Polska["Rok"]), linestyle="--", color="red", label="Linia trendu")
plt.title("Liczba przestępstw drogowych w latach 1999-2021 z linią trendu")
plt.xlabel("Rok")
plt.ylabel("Liczba przestępstw")
plt.xticks(rotation=45)
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.show()

####  Modelowanie statystyczne

In [ ]:
df_merged = pd.merge(df_bezrobocie_roczne, df_kr_Polska, on="Rok", how="inner")
df_merged = pd.merge(df_merged, df_gosp_Polska, on="Rok", how="inner")
df_merged = pd.merge(df_merged, df_dro_Polska, on="Rok", how="inner")
df_merged = df_merged.rename(columns={"Liczba przestępstw_x" : "Liczba przestępstw kryminalnych", "Liczba przestępstw_y" : "Liczba przestępstw gospodarczych", "Liczba przestępstw" : "Liczba przestępstw drogowych"})
df_merged['Rok'] = pd.to_datetime(df_merged['Rok'], format='%Y')
df_merged.set_index('Rok', inplace=True)

# Wybór kolumn do standaryzacji
kolumny = ["Bezrobocie [%]", "Liczba przestępstw kryminalnych","Liczba przestępstw drogowych", "Liczba przestępstw gospodarczych"]

scaler = StandardScaler()
df_merged[kolumny] = scaler.fit_transform(df_merged[kolumny])

X = df_merged[["Bezrobocie [%]"]]
y_kr = df_merged[["Liczba przestępstw kryminalnych"]]
y_dro = df_merged[["Liczba przestępstw drogowych"]]
y_gosp = df_merged[["Liczba przestępstw gospodarczych"]]


#Podział na zbiory treningowe i testowe
X_train, X_test, y_kr_train, y_kr_test = train_test_split(X, y_kr,test_size = 0.2, random_state=42)
_, _, y_dro_train, y_dro_test = train_test_split(X, y_dro,test_size = 0.2, random_state=42)
_, _, y_gosp_train, y_gosp_test = train_test_split(X, y_gosp,test_size = 0.2, random_state=42)

#Model regresji liniowej dla przestępczości kryminalnej
model_kr = LinearRegression()
model_kr.fit(X_train, y_kr_train)
y_kr_pred = model_kr.predict(X_test)

#Model regresji liniowej dla przestępczości drogowej
model_dro = LinearRegression()
model_dro.fit(X_train, y_kr_train)
y_dro_pred = model_dro.predict(X_test)

#Model regresji liniowej dla przestępczości gospodarcze
model_gosp = LinearRegression()
model_gosp.fit(X_train, y_kr_train)
y_gosp_pred = model_gosp.predict(X_test)


#Ocena modelu
mse_kr = mean_squared_error(y_kr_test, y_kr_pred)
r2_kr = r2_score(y_kr_test, y_kr_pred)

mse_kr = round(mse_kr,4)
r2_kr = round(r2_kr,4)

mse_dro = mean_squared_error(y_dro_test, y_dro_pred)
r2_dro = r2_score(y_dro_test, y_dro_pred)

mse_dro = round(mse_dro,4)
r2_dro = round(r2_dro,4)

mse_gosp = mean_squared_error(y_gosp_test, y_gosp_pred)
r2_gosp = r2_score(y_gosp_test, y_gosp_pred)

mse_gosp = round(mse_gosp,4)
r2_gosp = round(r2_gosp,4)

print(f"Regresja kryminalna: {mse_kr}, R^2 Score: {r2_kr}")
print(f"Regresja drogowa: {mse_dro}, R^2 Score: {r2_dro}")
print(f"Regresja gospodarcza: {mse_gosp}, R^2 Score: {r2_gosp}")


#Korelacja Pearsona
corr_kryminalne, _ = pearsonr(df_merged["Bezrobocie [%]"], df_merged["Liczba przestępstw kryminalnych"])
corr_drogowe, _ = pearsonr(df_merged["Bezrobocie [%]"], df_merged["Liczba przestępstw drogowych"])
corr_gospodarcze, _ = pearsonr(df_merged["Bezrobocie [%]"], df_merged["Liczba przestępstw gospodarczych"])

corr_kryminalne = round(corr_kryminalne, 4)
corr_drogowe = round(corr_drogowe, 4)
corr_gospodarcze = round(corr_gospodarcze, 4)

print(f"Korelacja Pearsona: {corr_kryminalne}")
print(f"Korelacja Pearsona: {corr_drogowe}")
print(f"Korelacja Pearsona: {corr_gospodarcze}")


# Podział danych na grupy
niskie_bezrobocie = df_merged[df_merged["Bezrobocie [%]"] < df_merged["Bezrobocie [%]"].median()]
wysokie_bezrobocie = df_merged[df_merged["Bezrobocie [%]"] >= df_merged["Bezrobocie [%]"].median()]

# Test t-Studenta dla przestępstw kryminalnych
t_stat_kryminalne, p_value_kryminalne = ttest_ind(niskie_bezrobocie["Liczba przestępstw kryminalnych"], wysokie_bezrobocie["Liczba przestępstw kryminalnych"])

t_stat_kryminalne = round(t_stat_kryminalne, 4)
p_value_kryminalne = round(p_value_kryminalne, 4)

print(f"T-statystyka - przestępstwa kryminalne: {t_stat_kryminalne}, p-wartość: {p_value_kryminalne}")

# Test t-Studenta dla przestępstw drogowych
t_stat_drogowe, p_value_drogowe = ttest_ind(niskie_bezrobocie["Liczba przestępstw drogowych"], wysokie_bezrobocie["Liczba przestępstw drogowych"])

t_stat_drogowe = round(t_stat_drogowe, 4)
p_value_drogowe = round(p_value_drogowe, 4)

print(f"T-statystyka - przestępstwa drogowe: {t_stat_drogowe}, p-wartość: {p_value_drogowe}")

# Test t-Studenta dla przestępstw gospodarczych
t_stat_gospodarcze, p_value_gospodarcze = ttest_ind(niskie_bezrobocie["Liczba przestępstw gospodarczych"], wysokie_bezrobocie["Liczba przestępstw gospodarczych"])

t_stat_gospodarcze = round(t_stat_gospodarcze, 4)
p_value_gospodarcze = round(p_value_gospodarcze, 4)

print(f"T-statystyka - przestępstwa gospodarcze: {t_stat_gospodarcze}, p-wartość: {p_value_gospodarcze}")

#### Przewidywanie przyszłych wartości - prognozowanie

1. Przestępstwa kryminalne

In [ ]:

# Test Dickeya-Fullera dla przestępstw kryminalnych

ts_kryminalne = df_merged["Liczba przestępstw kryminalnych"]

# Pierwsze różnicowanie
ts_kryminalne_diff = ts_kryminalne.diff().dropna()

result = adfuller(ts_kryminalne_diff)
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
for key, value in result[4].items():
    print("Critical Values:")
    print(f"   {key}, {value}")


In [ ]:
# Wykresy ACF i PACF dla przestępstw kryminalnych
fig, ax = plt.subplots(2, figsize=(12, 8))

plot_acf(df_merged["Liczba przestępstw kryminalnych"], ax=ax[0], lags=10)
plot_pacf(df_merged["Liczba przestępstw kryminalnych"], ax=ax[1], lags=10)

plt.show()

In [ ]:
# df_merged['Rok'] = pd.to_datetime(df_merged['Rok'], format='%Y')
# df_merged.set_index('Rok', inplace=True)

#Model Arima
model_kr = ARIMA(df_merged["Liczba przestępstw kryminalnych"], order=(1, 1, 1))
model_kr_fit = model_kr.fit()

# Prognozowanie
forecast = model_kr_fit.forecast(steps=5)

#Wykres prognoz
plt.figure(figsize=(10,6))
plt.title("Prognozowanie liczby przestępstw kryminalnych")
plt.plot(df_merged["Liczba przestępstw kryminalnych"], label="Historyczne dane")
plt.plot(pd.date_range(start=df_merged.index[-1], periods=6, freq="AS")[1:], forecast, label="Prognoza", color="red")
plt.legend()
plt.show()


2. Przestępstwa gospodarcze

In [ ]:
# Test Dickeya-Fullera dla przestępstw gospodarczych

ts_gospodarcze = df_merged["Liczba przestępstw gospodarczych"]

# Pierwsze różnicowanie
ts_gospodarcze_diff = ts_gospodarcze.diff().dropna()
# Drugie różnicowanie
ts_gospodarcze_diff2 = ts_gospodarcze_diff.diff().dropna()
# Trzecie różnicowanie
ts_gospodarcze_diff3 = ts_gospodarcze_diff2.diff().dropna()

result = adfuller(ts_gospodarcze_diff3)
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
for key, value in result[4].items():
    print("Critical Values:")
    print(f"   {key}, {value}")


In [ ]:
# Wykresy ACF i PACF dla przestępstw gospodarczych
fig, ax = plt.subplots(2, figsize=(12, 8))

plot_acf(df_merged["Liczba przestępstw gospodarczych"], ax=ax[0], lags=10)
plot_pacf(df_merged["Liczba przestępstw gospodarczych"], ax=ax[1], lags=10)

plt.show()

In [ ]:
#Model Arima
model_gosp = ARIMA(df_merged["Liczba przestępstw gospodarczych"], order=(1, 3, 2))
model_gosp_fit = model_gosp.fit()

# Prognozowanie
forecast = model_gosp_fit.forecast(steps=5)

# Wykres prognoz
plt.figure(figsize=(10,6))
plt.title("Prognozowanie liczby przestępstw gospodarczych")
plt.plot(df_merged["Liczba przestępstw gospodarczych"], label="Historyczne dane")
plt.plot(pd.date_range(start=df_merged.index[-1], periods=6, freq="AS")[1:], forecast, label="Prognoza", color="red")
plt.legend()
plt.show()

3. Przestępstwa drogowe

In [ ]:
# Test Dickeya-Fullera dla przestępstw drogowych

ts_drogowe = df_merged["Liczba przestępstw drogowych"]

# Pierwsze różnicowanie
ts_drogowe_diff = ts_drogowe.diff().dropna()

result = adfuller(ts_drogowe_diff)
print("ADF Statistic: %f" % result[0])
print("p-value: %f" % result[1])
for key, value in result[4].items():
    print("Critical Values:")
    print(f"   {key}, {value}")

In [ ]:
# Wykresy ACF i PACF dla przestępstw gospodarczych
fig, ax = plt.subplots(2, figsize=(12, 8))

plot_acf(df_merged["Liczba przestępstw drogowych"], ax=ax[0], lags=10)
plot_pacf(df_merged["Liczba przestępstw drogowych"], ax=ax[1], lags=10)

plt.show()

In [ ]:
#Model Arima
model_dro = ARIMA(df_merged["Liczba przestępstw drogowych"], order=(1, 1, 2))
model_dro_fit = model_dro.fit()

# Prognozowanie
forecast = model_dro_fit.forecast(steps=5)

# Wykres prognoz
plt.figure(figsize=(10,6))
plt.title("Prognozowanie liczby przestępstw drogowych")
plt.plot(df_merged["Liczba przestępstw drogowych"], label="Historyczne dane")
plt.plot(pd.date_range(start=df_merged.index[-1], periods=6, freq="YS")[1:], forecast, label="Prognoza", color="red")
plt.legend()
plt.show()

#### Analiza skupień

1. Przestępstwa kryminalne

In [ ]:
df_klasteryzacja = df_merged[["Bezrobocie [%]", "Liczba przestępstw kryminalnych", "Liczba przestępstw drogowych", "Liczba przestępstw gospodarczych"]]
df_klasteryzacja_scaled = scaler.fit_transform(df_klasteryzacja)

# Wybór optymalnej liczby klastrów za pomocą metody "łokcia"
inertia = []
for n in range(1, 11):
    kmeans = KMeans(n_clusters=n, random_state=42)
    kmeans.fit(df_klasteryzacja_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Metoda "łokcia" dla określenia optymalnej liczby klastrów')
plt.xlabel('Liczba klastrów')
plt.ylabel('Inercja')
plt.show()

# Klasteryzacja z optymalną liczbą klastrów
kmeans = KMeans(n_clusters=3, random_state=42)  # Zakładamy, że 3 klastry są optymalne
df_merged["Klaster"] = kmeans.fit_predict(df_klasteryzacja_scaled)

# Wykres wyników klasteryzacji
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x="Bezrobocie [%]", y="Liczba przestępstw kryminalnych", hue="Klaster", palette="viridis")
plt.title('Klasteryzacja K-means: Bezrobocie a przestępstwa kryminalne')
plt.show()

2. Przestępstwa gospodarcze

In [ ]:
df_klasteryzacja = df_merged[["Bezrobocie [%]", "Liczba przestępstw kryminalnych", "Liczba przestępstw drogowych", "Liczba przestępstw gospodarczych"]]
df_klasteryzacja_scaled = scaler.fit_transform(df_klasteryzacja)

inertia = []
for n in range(1, 11):
    kmeans = KMeans(n_clusters=n, random_state=42)
    kmeans.fit(df_klasteryzacja_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Metoda "łokcia" dla określenia optymalnej liczby klastrów')
plt.xlabel('Liczba klastrów')
plt.ylabel('Inercja')
plt.show()

kmeans = KMeans(n_clusters=3, random_state=42)  
df_merged["Klaster"] = kmeans.fit_predict(df_klasteryzacja_scaled)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x="Bezrobocie [%]", y="Liczba przestępstw gospodarczych", hue="Klaster", palette="viridis")
plt.title('Klasteryzacja K-means: Bezrobocie a przestępstwa gospodarcze')
plt.show()

3. Przestępstwa drogowe

In [ ]:
df_klasteryzacja = df_merged[["Bezrobocie [%]", "Liczba przestępstw kryminalnych", "Liczba przestępstw drogowych", "Liczba przestępstw gospodarczych"]]
df_klasteryzacja_scaled = scaler.fit_transform(df_klasteryzacja)

# Wybór optymalnej liczby klastrów za pomocą metody "łokcia"
inertia = []
for n in range(1, 11):
    kmeans = KMeans(n_clusters=n, random_state=42)
    kmeans.fit(df_klasteryzacja_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Metoda "łokcia" dla określenia optymalnej liczby klastrów')
plt.xlabel('Liczba klastrów')
plt.ylabel('Inercja')
plt.show()

kmeans = KMeans(n_clusters=3, random_state=42) 
df_merged["Klaster"] = kmeans.fit_predict(df_klasteryzacja_scaled)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x="Bezrobocie [%]", y="Liczba przestępstw drogowych", hue="Klaster", palette="viridis")
plt.title('Klasteryzacja K-means: Bezrobocie a przestępstwa drogowe')
plt.show()

4. Suma wszystkich przestępstw w roku

In [ ]:
# Sumowanie przestępstw
df_merged["Suma wszystkich przestępstw"] = df_merged["Liczba przestępstw drogowych"] + df_merged["Liczba przestępstw kryminalnych"] + df_merged["Liczba przestępstw gospodarczych"]

# Przygotowanie danych do klasteryzacji
df_klasteryzacja = df_merged[["Suma wszystkich przestępstw"]].copy()
df_klasteryzacja['Rok'] = df_merged.index.year  # Dodanie roku jako kolumny
scaler = StandardScaler()
df_klasteryzacja_scaled = scaler.fit_transform(df_klasteryzacja)

# Klasteryzacja K-means
kmeans = KMeans(n_clusters=3, random_state=42)
df_merged['Klaster'] = kmeans.fit_predict(df_klasteryzacja_scaled)

# Wizualizacja klastrów
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_merged, x=df_merged.index.year, y="Suma wszystkich przestępstw", hue="Klaster", palette="viridis")
plt.title('Klasteryzacja K-means: Rok a liczba przestępstw')
plt.xlabel("Rok")
plt.ylabel("Suma wszystkich przestępstw")
plt.show()